## Point calculator of the preindustrial DIC for tests 

Is meant to replicate what the large loops do for a single point - using the C*/AOU method:
-     1) assumption of age from sigma-theta (which comes from salinity (g/kg) and temperature (CT) 
-     2) assumption of pycnal-witnessed-co2 from lawdome_maunaloa.csv (single pco2 value per year)
-     3) assumption of AOU from in-situ oxygen (umol/L) and o2 solubility from gsw tolbox assumption; assumption of stoichiometric C:0 ratio of -117/170 from Gruber > this can give us preformed pco2
- 4) assumption of preformed-pco2 with potential temperature corresp. to conservative temperature, depth = 0 
- 5) disequilibrium calculated between pycnal-witnessed-co2 and preformed-pco2
- 6) disequilibrium kept constant with preindustrial co2 (284 uatm) - with disequilibrium + 284 uatm, we can get preindustrial preformed pco2
- 7) from preindustrial preformed pco2 and present-day everything else and DEPTH ZERO!!!! we can get preindustrial preformed DIC.
- 8) this then gives us a delta DIC (pref.DIC-preind.pref.DIC) that we can use to calculate preindustrial dic
    
Requires/usage:

        toy_preind_DIC_calculator(DIC, TA, OXY, depth,sal, temp)
        
Calls:
        co2_from_year(year)
        find_DIC_corresp_to_pco2(tsal, ttemp, tpco2, tta, pres_atm, depth_this)
        oned_moxy(tsal, ttemp, tdic, tta, pres_atm, depth_this)

In [47]:
def find_nearest(array, value):
    
    import numpy as np
    
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx, array[idx]

def co2_from_year(year):
    
    import pandas as pd
    '''takes a value for a year, converts year to int,
    and finds appropriate co2 values  from pandas lookup table. 
    if year < 1832, value is for year 1832, if year > 2018, value is for year 2018'''
    co2_rec = pd.read_csv('lawdome_maunaloa.csv') 
    whole_year = int(year)
    
    if whole_year >= 2018:
        whole_year = 2018     
        #('year > 2018, using value for 2018')       
    if whole_year <= 1832:
        whole_year = 1832
        #('year < 1832, using value for 1832')

    match = (co2_rec['YEAR'] == whole_year) 
    atmco2 = co2_rec['PPMCO2'][match]
    t_co2 = atmco2.values[0]
    return t_co2

def find_DIC_corresp_to_pco2(tsal, ttemp, tpco2, tta, pres_atm, depth_this):
    
    import numpy as np
    import mocsy
    import gsw
    
    steps = 10000
    tsal_r = np.zeros([steps])
    tsal_r[:] = tsal
    #convert to psu
    tsal_r_psu = tsal_r*35/35.16504
    
    ttemp_r = np.zeros([steps])
    ttemp_r[:] = ttemp
    #convert temperature to potential temperature
    ttemp_r_pot = gsw.pt_from_CT(tsal_r,ttemp_r)
    tta_r = np.zeros([steps])
    tta_r[:] = tta * 1e-3
    tpres_r = np.zeros([steps])
    tpres_r[:] = pres_atm
    depth_r = np.zeros([steps])
    depth_r[:] = depth_this
    tzero = np.zeros([steps])
    tlat = np.zeros([steps])
    tlat[:]=50
    
    end_d = 2400
    start_d = 600
    intvl = (end_d - start_d)/steps
    tdic_r = np.arange(start_d,end_d-0.1,intvl) * 1e-3
    #change to take potential temperature
    response_tup = mocsy.mvars(temp=ttemp_r_pot, sal=tsal_r_psu, alk=tta_r, dic=tdic_r, 
                       sil=tzero, phos=tzero, patm=tpres_r, depth=depth_r, lat=tzero, 
                        optcon='mol/m3', optt='Tpot', optp='m',
                        optb = 'l10', optk1k2='m10', optkf = 'dg', optgas = 'Pinsitu')
    pH,pco2,fco2,co2,hco3,co3,OmegaA,OmegaC,BetaD,DENis,p,Tis = response_tup    
    
    diffmat = pco2 - tpco2
    idx, ans = find_nearest( diffmat,0 )
    
    if ans> 2:
        print('Danger, pco2 found >2 uatm from pco2 given')
#     print(idx)
#     print('difference between real pco2 and pco2 from calc. dic: ',ans)
#     print('DIC found this way:', tdic_r[idx]*1e3)
    fin_dic = tdic_r[idx]*1e3
    
    return fin_dic

In [48]:
def oned_moxy(tsal, ttemp, tdic, tta, pres_atm, depth_this):
    
    '''Just a mocsy wrapper - here we are using potential temp and\
    whatever depth we enter, salinity as psu '''
    import sys
    sys.path.append('/data/tjarniko/mocsy')
    import mocsy
    import numpy as np
    import gsw

    tsra = np.ravel(tsal)
    ttera = np.ravel(ttemp)
    #convert cons. temperature to potential temperature
    ttera_pot = gsw.pt_from_CT(tsra,ttera)
    
    ttara = np.ravel(tta) * 1e-3
    tdra = np.ravel(tdic) * 1e-3
    tzero = np.zeros_like(tsra)
    tpressure = np.zeros_like(tsra)
    #tdepth = np.zeros_like(tsra)
    tpressure[:] = pres_atm
    tdepth = np.ravel(depth_this)
    tzero = tpressure * 0 
        
    tsra_psu = tsra*35/35.16504
    #ttera_is = gsw.t_from_CT(tsra,ttera,tzero)

    response_tup = mocsy.mvars(temp=ttera_pot, sal=tsra_psu, alk=ttara, dic=tdra, 
                       sil=tzero, phos=tzero, patm=tpressure, depth=tdepth, lat=tzero, 
                        optcon='mol/m3', optt='Tpot', optp='m',
                        optb = 'l10', optk1k2='m10', optkf = 'dg', optgas = 'Pinsitu')
    pH,pco2,fco2,co2,hco3,co3,OmegaA,OmegaC,BetaD,DENis,p,Tis = response_tup

    
    return pH, OmegaA, pco2

In [49]:


def toy_preind_DIC_calculator(DIC, TA, O2, depth,sal, temp):
    
    
    import gsw
    import numpy as np
    
    # TJSJ calculate sigma0 
    sigma0 = gsw.sigma0(sal,temp)
    
    
    ## TJSJ find pycnal witnessed co2 (age from relationship with sigma0)
    params0 = 0.1301889490932413
    params1 = 3.8509914822057825
    params2 = 8.301166081413104 #change to 2015 since model year is 2015

    pycnal_last_at_surface = \
    2015 - (params0 *np.exp(-params1*(25.15-sigma0))+params2)
    pycnal_witnessed_atm_co2 = co2_from_year(pycnal_last_at_surface)
    
    # TJSJ calculate AOU, for which we need o2 solubility in umol/L
    osol = gsw.O2sol(sal,temp,0,-125,50)
    #convert osol to umol/L
    osol_umolL = osol*(1000/(1000+sigma0))
    AOU = osol_umolL - O2
    
    # TJSJ get stoichiometric AOU using (117/170)
    AOU_stoich = AOU * (117/170)
    
    # TJSJ get preformed DIC!
    preformed_DIC = DIC - AOU_stoich
    
    #TJSJ get preformed pco2 - the 0 is DEPTH the 1 is atmospheric pressure - and also disequilibrium
    
    pHr, OmAr, pco2r = \
    oned_moxy(sal, temp, preformed_DIC, TA, 1, 0)
    preformed_pco2 = pco2r
    diseqPCO2 = preformed_pco2 - pycnal_witnessed_atm_co2

    #get preindustrial preformed pco2
    PIpreformed_pco2 = diseqPCO2 + 284
    #
    
    #get preformed DIC
    PIpreformed_DIC = find_DIC_corresp_to_pco2(sal, temp, PIpreformed_pco2, TA, 1, 0)

    
    deltaDIC = preformed_DIC - PIpreformed_DIC
    
    finalPI_DIC = DIC - deltaDIC
    
    return sigma0, pycnal_last_at_surface, pycnal_witnessed_atm_co2, AOU, preformed_DIC, preformed_pco2, PIpreformed_pco2, PIpreformed_DIC, finalPI_DIC



## TEST CASE

In [50]:
print('~~~INPUTS (all model units)~~~~')
DIC = 2150; TA = 2200; OXY = 200; depth = 0; sal = 34;  temp = 7
print(f'DIC: {DIC}, TA: {TA}, OXY: {OXY}')
print(f'sal: {sal}, temp: {temp}')
print(f'depth, m (DOESNT MATTER FOR THIS CALCULATION! AS WE HAVE LEARNT TO OUR DISMAY): {depth})')
print('')

sigma0, pycnal_last_at_surface, pycnal_witnessed_atm_co2,\
AOU, preformed_DIC, preformed_pco2, PIpreformed_pco2,\
PIpreformed_DIC, finalPI_DIC = toy_preind_DIC_calculator(DIC, TA, OXY, depth,sal, temp)

print('~~~OUTPUTS (all model units)~~~~')
print(f'sigma0: {sigma0}')
print(f'pycnal last at surface: {pycnal_last_at_surface}')
print(f'pycnal_witnessed_atm_co2: {pycnal_witnessed_atm_co2}')
print(f'AOU: {AOU}')
print(f'preformed_DIC: {preformed_DIC}')
print(f'preformed_pco2: {preformed_pco2}')
print(f'PIpreformed_pco2: {PIpreformed_pco2}')
print(f'PIpreformed_DIC: {PIpreformed_DIC}')
print(f'finalPI_DIC: {finalPI_DIC}')

~~~INPUTS (all model units)~~~~
DIC: 2150, TA: 2200, OXY: 200
sal: 34, temp: 7
depth, m (DOESNT MATTER FOR THIS CALCULATION! AS WE HAVE LEARNT TO OUR DISMAY): 0)

~~~OUTPUTS (all model units)~~~~
sigma0: 26.507752150228725
pycnal last at surface: 1982.412263588211
pycnal_witnessed_atm_co2: 341.45
AOU: 88.36319644392893
preformed_DIC: 2089.185329506237
preformed_pco2: [516.11697165]
PIpreformed_pco2: [458.66697165]
PIpreformed_DIC: 2072.759999999591
finalPI_DIC: 2133.574670493354
